#### Code to generate predictions using a weighted SVM; only PD-L1 marker is used


In [67]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

from tqdm import tqdm
from itertools import chain


from scipy import ndimage

from sklearn.manifold import TSNE

from scipy.stats import zscore

from skimage.color import rgb2gray

from matplotlib.patches import Polygon

from skimage import io

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import seaborn as sns

import scipy.spatial

from scipy.spatial import ConvexHull, convex_hull_plot_2d


from scipy.spatial import distance


import seaborn as sns


import matplotlib.image as mpimg 
import time as time

import matplotlib.colors as colors 

from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

import itertools

In [ ]:

path_python = './spatial_cells_SVM_Lung1/'


dpi_set = 100

num_colour = 30
cm = plt.cm.get_cmap('tab20', num_colour)

num_comp = 8

count = 0

pat_num = []

In [70]:
NSCLC_CM_FOV = []
NSCLC_CM_list = []
cells_per_FOV = []
cent_per_FOV = []
stitched_image_list = []
label_im_st_list = []
X_2d_list = []

In [73]:
# read in files and weight according to quadrat weight means

cell_counts = []
fname_spat = []

pat_ids_xy = [33, 34, 36, 37, 41, 43, 44, 47, 54]

weights_per_patient = [0.44,
0.504,
0.523,
0.53,
0.504,
0.51,
0.55,
0.47,
0.517]

data_CM = np.zeros((1,6)) #will have all the Count matrices consolidated
prog_mat = np.zeros((1))
accu_mat = np.zeros((1))

pat_id = []
resp_categ = []
num_cells_categ = []  

#data_CM_spat = np.zeros((1,20))

for fname in os.listdir(path_python): 
    df_spat = pd.read_csv(os.path.join(path_python+fname))
   
    
    print(df_spat.shape)
    print(fname)
    data_spat1 = np.array(df_spat )
    data_spat = np.array(df_spat )

    name = fname.split("_")
    

    ## weighting the data
    id_xy = np.array(np.where(pat_ids_xy==np.int64(name[1]))).flatten()
    temp_xy = data_spat[:,0:6]
    data_spat[:,0:6]=weights_per_patient[id_xy[0]]*temp_xy 
    print(weights_per_patient[id_xy[0]])

    data_CM = np.concatenate((data_CM,data_spat[:,0:6]),axis=0)
    pat_id.append(np.repeat(name[1],data_spat.shape[0]))
    resp_categ.append(np.repeat(name[2],data_spat.shape[0]))
    num_cells_categ.append(data_spat.shape[0])
    fname_spat.append(fname)
    

    print(name[1])
    print(name[2])


(1340, 7)
pat_47_sd_pre_40994_16041_spat_distance_CM_post1.csv
0.47
47
sd
(1206, 7)
pat_44_sd_pre_41191_15079_spat_distance_CM_post1.csv
0.55
44
sd
(1237, 7)
pat_54_pd_pre_50076_15190_spat_distance_CM_post1.csv
0.517
54
pd
(2064, 7)
pat_34_sd_pre_42959_16846_spat_distance_CM_post1.csv
0.504
34
sd
(1262, 7)
pat_47_sd_pre_40503_15806_spat_distance_CM_post1.csv
0.47
47
sd
(1407, 7)
pat_41_pd_pre_41430_16100_0_spat_distance_CM_post1.csv
0.504
41
pd
(1470, 7)
pat_44_sd_pre_47289_12844_spat_distance_CM_post1.csv
0.55
44
sd
(1464, 7)
pat_36_pd_pre_43257_16035_spat_distance_CM_post1.csv
0.523
36
pd
(1936, 7)
pat_34_sd_pre_40842_18076_spat_distance_CM_post1.csv
0.504
34
sd
(1445, 7)
pat_44_sd_pre_41823_12896_spat_distance_CM_post1.csv
0.55
44
sd
(1426, 7)
pat_47_sd_pre_40235_14110_spat_distance_CM_post1.csv
0.47
47
sd
(1143, 7)
pat_54_pd_pre_50386_13738_spat_distance_CM_post1.csv
0.517
54
pd
(1713, 7)
pat_33_sd_pre_52082_11436_spat_distance_CM_post1.csv
0.44
33
sd
(1030, 7)
pat_33_sd_pre_47625_

In [75]:
len(pat_num_list)

64648

In [76]:
resp_categ_list = [item for sublist in resp_categ for item in sublist]

In [77]:
len(resp_categ_list)

64648

In [78]:
data_CM.shape

(64649, 6)

In [79]:
data_CM = np.delete(data_CM, (0), axis=0)

#remove rows that have 0 rowsum

row_sums_CM = data_CM.sum(axis=1,keepdims=True)
rows_remove = np.where(row_sums_CM==0)
data_CM = np.delete(data_CM,(rows_remove),axis=0)


In [80]:
data_CM.shape

(62923, 6)

In [81]:
# rows remove if not needed

resp_categ_list_flat = np.delete(np.array(resp_categ_list).flatten(),(rows_remove),axis=0)
pat_num_list_flat = np.delete(np.array(pat_num_list).flatten(),(rows_remove),axis=0)



In [82]:
print(pat_num_list_flat.shape)
print(resp_categ_list_flat.shape)

(62923,)
(62923,)


In [83]:


M =  data_CM #np.log(data_CM+0.1) #zscore(data_CM+0.1,axis=1)

M_upd = M

num_cells = M.shape[0]
num_genes = M.shape[1]

In [84]:
np.unique(resp_categ_list_flat, return_counts=True)

(array(['pd', 'sd'], dtype='<U2'), array([21040, 41883]))

##### PD-L1 marker

In [98]:
M_upd.shape

(62923, 6)

In [99]:
### pulling out the pdl1 data
data_pred=M_upd[:,3]
data_pred.shape

(62923,)

In [100]:
data_pred = np.reshape(data_pred,[-1,1])

In [101]:
data_pred.shape

(62923, 1)

In [102]:
data_target = np.array(resp_categ_list_flat).flatten()
dd = data_target
sd=np.where(data_target=='sd')
dd[sd]=1
pd=np.where(data_target=='pd')
dd[pd]=0

data_target.shape

(62923,)

In [103]:
dd =np.int64(dd)
dd
data_target = dd

In [104]:
print(type(data_target))
print(data_target)

<class 'numpy.ndarray'>
[1 1 1 ... 1 1 1]


In [105]:
y_marker = ['CD3', 'PD1', 'CD8', 'PDL1', 'FoxP3', 'PanCK', 'DAPI']

#### SVM prediction engine

In [110]:
from sklearn.model_selection import LeavePOut, LeaveOneOut
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [111]:
pat_list = np.int64(np.array(pat_num_list_flat).flatten())

In [116]:
## choose between loo or lpo

from sklearn import svm, metrics
from sklearn.metrics import roc_curve, auc

#all
svm_acc=[]
svm_pred=[]
svm_recall=[]
svm_coef =[]
svm_f1_score=[]
svm_roc_auc_score =[]
svm_train_size = []
svm_test_size = []

#sum
svm_pred_sum=[]
svm_recall_sum=[]
svm_f1_score_sum=[]
svm_auc_train=[]
svm_auc_test=[]

# sd
svm_acc_sd = []
svm_pred_sd=[]
svm_recall_sd=[]
svm_f1_score_sd=[]
svm_auc_train_sd=[]
svm_auc_test_sd=[]

#pd
svm_acc_pd = []
svm_pred_pd=[]
svm_recall_pd=[]
svm_f1_score_pd=[]
svm_auc_train_pd=[]
svm_auc_test_pd=[]

train_fpr_list=[]
test_fpr_list = []
test_tpr_list = []
train_tpr_list = []


X = np.ones(8) #without #43
lpo = LeavePOut(p=1)
loo = LeaveOneOut()
all_pat = [33, 34, 36, 37, 41, 44, 47, 54] #[33, 34, 36, 37, 41, 43, 44, 47, 54]
count3 = 0


all_y = []
all_probs=[]


for train, test in lpo.split(X): ## loo or lpo...
    print("%s %s" % (train, test))
    
    #p_pred_train=[47,37,36,41,54,33,44]
    rows1=[]
    for ii in range(len(train)):
        #print(ii)
        rows1.append(np.array(np.where(pat_list==all_pat[train[ii]])).flatten())
    
    flatlist = [element for sublist in rows1 for element in sublist]
    rows1=flatlist

    #p_pred_test=[43,34]
    rows2=[]
    for ii in range(len(test)):
        print('**** Patient left out is ',all_pat[test[ii]])
        rows2.append(np.array(np.where(pat_list==all_pat[test[ii]])).flatten())
    flatlist=[element for sublist in rows2 for element in sublist]
    rows2=flatlist 
    
    
    

    X_train= data_pred[rows1,:]
    X_test = data_pred[rows2,:]
    #[33, 34, 36, 37, 41, 43, 44, 47, 54]


    y_train= data_target[rows1]
    y_test = data_target[rows2]
    print('X_train shape is: ',X_train.shape)
    print('X_test shape is: ', X_test.shape)
    print('y_train shape is: ',y_train.shape)
    print('y_test shape is: ',y_test.shape)
    
    
    # generating the model

    #Create a svm Classifier
    clf = svm.SVC(kernel='linear') # Linear Kernel
    
    
    #Train the model using the training sets
    clf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    
    #evaluating the model 
    #Import scikit-learn metrics module for accuracy calculation
    
    # for the ROC
    y_train_pred = clf.decision_function(X_train)    
    y_test_pred = clf.decision_function(X_test)


    
    # Model Accuracy: how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    # Model Precision: what percentage of positive tuples are labeled as such?
    print("Precision:",metrics.precision_score(y_test, y_pred,zero_division=0,average=None))

    # Model Recall: what percentage of positive tuples has the model correctly identified as positive?
    print("Recall:",metrics.recall_score(y_test, y_pred,zero_division=0,average=None))


    #F1 score
    print("F1 score:", str(metrics.f1_score(y_test, y_pred,zero_division=0,average=None)))



    print("coefficients :", str(clf.coef_[0])) #Aug 8th 2024


    ####
    ##### end of JUne 27th 2024 code add
    

    svm_acc.append(metrics.accuracy_score(y_test, y_pred))
    svm_pred.append(metrics.precision_score(y_test, y_pred,zero_division=0))
    svm_recall.append(metrics.recall_score(y_test, y_pred,zero_division=0))
    svm_coef.append(clf.coef_[0])
    svm_f1_score.append(metrics.f1_score(y_test, y_pred))
    svm_train_size.append(X_train.shape[0])
    svm_test_size.append(X_test.shape[0])

    train_tpr_list.append(train_tpr)
    test_tpr_list.append(test_tpr)
    train_fpr_list.append(train_fpr)
    test_fpr_list.append(test_fpr)
    
    count3= count3+1


    # sum for for the overall numbers
    svm_pred_sum.append(np.sum(metrics.precision_score(y_test, y_pred,zero_division=0,average=None)))
    svm_recall_sum.append(np.sum(metrics.recall_score(y_test, y_pred,zero_division=0,average=None)))
    svm_f1_score_sum.append(np.sum(metrics.f1_score(y_test, y_pred,average=None)))

   ## collecting stats per fold
    
    if (all_pat[test[ii]] in [33, 34, 37, 47,44]):
        svm_acc_sd.append(metrics.accuracy_score(y_test, y_pred))
        if (len(metrics.precision_score(y_test, y_pred,zero_division=0,average=None))==1):
            svm_pred_sd.append(metrics.precision_score(y_test, y_pred,zero_division=0,average=None)[0])
            svm_recall_sd.append(metrics.recall_score(y_test, y_pred,zero_division=0,average=None)[0])
            svm_f1_score_sd.append(metrics.f1_score(y_test, y_pred,average=None)[0])
            
        elif (len(metrics.precision_score(y_test, y_pred,zero_division=0,average=None))>1):
            svm_pred_sd.append(metrics.precision_score(y_test, y_pred,zero_division=0,average=None)[1])
            svm_recall_sd.append(metrics.recall_score(y_test, y_pred,zero_division=0,average=None)[1])
            svm_f1_score_sd.append(metrics.f1_score(y_test, y_pred,average=None)[1])

    else:
        svm_acc_pd.append(metrics.accuracy_score(y_test, y_pred))
        svm_pred_pd.append(metrics.precision_score(y_test, y_pred,zero_division=0,average=None)[0])
        svm_recall_pd.append(metrics.recall_score(y_test, y_pred,zero_division=0,average=None)[0])
        svm_f1_score_pd.append(metrics.f1_score(y_test, y_pred,average=None)[0])

        


[1 2 3 4 5 6 7] [0]
**** Patient left out is  33
X_train shape is:  (52650, 1)
X_test shape is:  (9783, 1)
y_train shape is:  (52650,)
y_test shape is:  (9783,)
Accuracy: 1.0
Precision: [1.]
Recall: [1.]
F1 score: [1.]
coefficients : [5.67515321e-07]
[0 2 3 4 5 6 7] [1]
**** Patient left out is  34
X_train shape is:  (52408, 1)
X_test shape is:  (10025, 1)
y_train shape is:  (52408,)
y_test shape is:  (10025,)
Accuracy: 1.0
Precision: [1.]
Recall: [1.]
F1 score: [1.]
coefficients : [7.66116494e-07]
[0 1 3 4 5 6 7] [2]
**** Patient left out is  36
X_train shape is:  (55648, 1)
X_test shape is:  (6785, 1)
y_train shape is:  (55648,)
y_test shape is:  (6785,)
Accuracy: 0.0
Precision: [0. 0.]
Recall: [0. 0.]
F1 score: [0. 0.]
coefficients : [4.20115248e-07]
[0 1 2 4 5 6 7] [3]
**** Patient left out is  37
X_train shape is:  (58207, 1)
X_test shape is:  (4226, 1)
y_train shape is:  (58207,)
y_test shape is:  (4226,)
Accuracy: 1.0
Precision: [1.]
Recall: [1.]
F1 score: [1.]
coefficients : [7

In [117]:
#sum
print("%0.4f accuracy with a standard deviation of %0.4f" % (np.array(svm_acc).mean(), np.array(svm_acc).std()))
print("%0.4f precision with a standard deviation of %0.4f" % (np.array(svm_pred_sum).mean(), np.array(svm_pred_sum).std()))
print("%0.4f recall with a standard deviation of %0.4f" % (np.array(svm_recall_sum).mean(), np.array(svm_recall_sum).std()))
print("%0.4f f1 score with a standard deviation of %0.4f" % (np.array(svm_f1_score_sum).mean(), np.array(svm_f1_score_sum).std()))


0.6250 accuracy with a standard deviation of 0.4841
0.6250 precision with a standard deviation of 0.4841
0.6250 recall with a standard deviation of 0.4841
0.6250 f1 score with a standard deviation of 0.4841


In [118]:
#SD 
print("%0.4f accuracy with a standard deviation of %0.4f" % (np.array(svm_acc_sd).mean(), np.array(svm_acc_sd).std()))
print("%0.4f precision with a standard deviation of %0.4f" % (np.array(svm_pred_sd).mean(), np.array(svm_pred_sd).std()))
print("%0.4f recall with a standard deviation of %0.4f" % (np.array(svm_recall_sd).mean(), np.array(svm_recall_sd).std()))
print("%0.4f f1 score with a standard deviation of %0.4f" % (np.array(svm_f1_score_sd).mean(), np.array(svm_f1_score_sd).std()))


1.0000 accuracy with a standard deviation of 0.0000
1.0000 precision with a standard deviation of 0.0000
1.0000 recall with a standard deviation of 0.0000
1.0000 f1 score with a standard deviation of 0.0000


In [120]:
#PD
print("%0.4f accuracy with a standard deviation of %0.4f" % (np.array(svm_acc_pd).mean(), np.array(svm_acc_pd).std()))
print("%0.4f precision with a standard deviation of %0.4f" % (np.array(svm_pred_pd).mean(), np.array(svm_pred_pd).std()))
print("%0.4f recall with a standard deviation of %0.4f" % (np.array(svm_recall_pd).mean(), np.array(svm_recall_pd).std()))
print("%0.4f f1 score with a standard deviation of %0.4f" % (np.array(svm_f1_score_pd).mean(), np.array(svm_f1_score_pd).std()))


0.0000 accuracy with a standard deviation of 0.0000
0.0000 precision with a standard deviation of 0.0000
0.0000 recall with a standard deviation of 0.0000
0.0000 f1 score with a standard deviation of 0.0000
